In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [6]:
bnf_1min = pd.read_csv("../data/sensex__30m.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [7]:
bnf_1min.tail()

,time,open,high,low,close,MA,MA.1,EMA,datetime
22841,2025-02-12T12:15:00+05:30,76126.6600,76247.4500,75982.6200,76129.9900,76495.4304,76727.1207,77042.2789,2025-02-12 12:15:00+05:30
22842,2025-02-12T12:45:00+05:30,76136.9700,76369.9100,76136.9700,76342.4500,76457.6122,76694.7215,77021.9940,2025-02-12 12:45:00+05:30
22843,2025-02-12T13:15:00+05:30,76341.8200,76459.7200,76229.3600,76350.7600,76422.6448,76665.6848,77002.5379,2025-02-12 13:15:00+05:30
22844,2025-02-12T13:45:00+05:30,76351.3700,76351.3700,75926.0000,76051.8000,76378.9081,76634.0367,76974.9803,2025-02-12 13:45:00+05:30
22845,2025-02-12T14:15:00+05:30,76042.7700,76101.9100,75976.7700,75998.0800,76335.6515,76590.3193,76946.6644,2025-02-12 14:15:00+05:30


In [8]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [9]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2019, 5, 23),
 datetime.date(2022, 9, 26),
 datetime.date(2023, 11, 30),
 datetime.date(2024, 9, 5),
 datetime.date(2023, 11, 3),
 datetime.date(2019, 12, 19),
 datetime.date(2022, 8, 22),
 datetime.date(2022, 4, 28),
 datetime.date(2023, 7, 26),
 datetime.date(2020, 6, 5),
 datetime.date(2023, 9, 28),
 datetime.date(2023, 11, 1),
 datetime.date(2023, 12, 11),
 datetime.date(2021, 3, 17),
 datetime.date(2023, 2, 16),
 datetime.date(2024, 1, 20),
 datetime.date(2024, 5, 23),
 datetime.date(2021, 4, 19),
 datetime.date(2019, 5, 27),
 datetime.date(2023, 9, 13),
 datetime.date(2024, 6, 10),
 datetime.date(2019, 8, 6),
 datetime.date(2021, 12, 10),
 datetime.date(2023, 2, 21),
 datetime.date(2021, 12, 22),
 datetime.date(2020, 7, 23),
 datetime.date(2020, 7, 20),
 datetime.date(2018, 5, 10),
 datetime.date(2020, 12, 16),
 datetime.date(2024, 8, 27),
 datetime.date(2018, 6, 11),
 datetime.date(2020, 7, 24),
 datetime.date(2020, 12, 2),
 datetime.date(2024, 8, 16),
 datetime.d

In [10]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
    df = df.rename(column_mapping)

    return df

In [11]:
PORTFOLIO_VALUE = 12_50_000 # 10 Lacs
# RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.0001
LEVERAGE_ = 5

In [12]:
def calculate_signals(df, ema_length, x_days, pct):
    # Calculate X-day High/Low and 20 EMA
    df['X_High'] = df['high'].rolling(x_days).mean()
    df['X_Low'] = df['low'].rolling(x_days).mean()
    df['EMA_20'] = df['close'].ewm(span=ema_length, adjust=False).mean()
    
    # Trend conditions
    df['EMA_Trend'] = df['EMA_20'] > df['EMA_20'].shift(1)
    
    # Signal conditions
    df['Buy_Signal'] = (
        (df['close'] > df['X_High'].shift(1) * (1 + (pct / 100))) & 
        (df['EMA_Trend']) & 
        (df['close'] > df['EMA_20'])
    )
    
    return df

In [13]:
def backtest3(df):
    # print(df.to_string())
    # return
    df.reset_index(drop=True, inplace=True)
    # Variables for long trades
    long_position = 0  # 0 = no position, 1 = long
    long_entry_price = 0
    long_entry_date = None
    long_trades = []
    tradebook = pd.DataFrame()
    tradebook_long = pd.DataFrame()
    # tradebook_short = pd.DataFrame()
    long_trailing_stop = None
    # Variables for short trades
    # short_position = 0  # 0 = no position, -1 = short
    # short_entry_price = 0
    # short_entry_date = None
    # short_trades = []
    # short_trailing_stop = None

    can_add_long = False
    # can_add_short = False

    trade_number = 0
    lock_initial_sl = False
    first_sl = 0

    lock_initial_sl_2 = False
    first_sl_2 = 0

    pyramid_number = 0
    can_pyramid = True
    pyramid_high = 0

    for i in range(1, len(df)):
        # Entry signals for long trades
        long_trailing_stop = df.loc[i, 'EMA_20']
        # short_trailing_stop = df.loc[i, 'EMA_20']
        
        if df.loc[i, 'Buy_Signal'] and long_position == 0:
            long_position = 1
            long_entry_price = df.loc[i, 'close']
            long_entry_date = df.loc[i, 'datetime']
            long_trailing_stop = df.loc[i, 'EMA_20']
            long_initial_sl = df.loc[i, 'X_Low']

            tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
                'Trade No.': trade_number,
                'Entry_Date': df.loc[i, 'datetime'],
                'Exit_Date': None,
                'Trade_Type': 'Long',
                'Entry_Price': df.loc[i, 'close'],
                'Previous MA Value': df.loc[i-1, 'X_High'],
                'Initial SL': long_initial_sl,
                'Exit_Price': None,
                # 'Profit/Loss': None  # Filled when exited
            }])], ignore_index=True)

            if not lock_initial_sl:
                first_sl = long_initial_sl
                lock_initial_sl = True

            can_pyramid = True
            pyramid_number = 0
            pyramid_high = 0
            continue
                
        # Exit or add to long position
        if long_position == 1:

            
            # y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
            # y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
            # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
            if df.loc[i, 'low'] <= long_initial_sl:
                if df.loc[i, 'open'] <= long_initial_sl:
                    tradebook_long.loc[
                        (tradebook_long['Trade No.'] == trade_number),
                        ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    ] = [df.loc[i, 'datetime'] , df.loc[i, 'open'], 'Gap']
                    long_position = 0
                    can_add_long = False
                    trade_number += 1
                    lock_initial_sl = False
                    first_sl = 0
                    can_pyramid = False
                    pyramid_number = 0
                    pyramid_high = 0
                    # if df.loc[i, 'high'] >= long_initial_sl:
                    #     tradebook_long.loc[
                    #         (tradebook_long['Trade No.'] == trade_number),
                    #         ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    #     ] = [df.loc[i, 'datetime'] , long_initial_sl, 'Initial SL Hit']
                    #     long_position = 0
                    #     can_add_long = False
                    #     trade_number += 1
                    #     lock_initial_sl = False
                    #     first_sl = 0
                    #     can_pyramid = False
                    #     pyramid_number = 0
                    #     pyramid_high = 0
                    # elif df.loc[i, 'high'] < long_initial_sl:
                    #     tradebook_long.loc[
                    #         (tradebook_long['Trade No.'] == trade_number),
                    #         ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    #     ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'], 'Gap Open-Close Below ISL']
                    #     long_position = 0
                    #     can_add_long = False
                    #     trade_number += 1
                    #     lock_initial_sl = False
                    #     first_sl = 0
                    #     can_pyramid = False
                    #     pyramid_number = 0
                    #     pyramid_high = 0
                else:
                    tradebook_long.loc[
                        (tradebook_long['Trade No.'] == trade_number),
                        ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    ] = [df.loc[i, 'datetime'] , long_initial_sl, 'Initial SL Hit']
                    long_position = 0
                    can_add_long = False
                    trade_number += 1
                    lock_initial_sl = False
                    first_sl = 0
                    can_pyramid = False
                    pyramid_number = 0
                    pyramid_high = 0
            
            elif df.loc[i, 'close'] <= long_trailing_stop:
                tradebook_long.loc[
                    (tradebook_long['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'Trailing SL Hit']
                long_position = 0
                can_add_long = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0

            # elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
            #     # Previous Y Candle Low Breached, confirmation found. Add position on high break
            #     can_add_long = True

            # if can_add_long and long_position and can_pyramid:
            #     if df.loc[i, 'high'] >= y_days_high and pyramid_number < pyr_num:
            #         tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
            #             'Trade No.': trade_number,
            #             'Entry_Date': df.loc[i, 'datetime'],
            #             'Exit_Date': None,
            #             'Trade_Type': 'Add_Long',
            #             'Entry_Price': y_days_high,
            #             'Initial SL': first_sl,
            #             'Exit_Price': None,
            #             # 'Profit/Loss': None  # Filled when exited
            #         }])], ignore_index=True)
            #         can_add_long = False
            #         pyramid_number += 1
            #         can_pyramid = True
    
    tradebook = pd.concat([tradebook_long, tradebook], ignore_index=True)
    return tradebook


In [14]:
def backtest(df):
    # print(df.to_string())
    # return
    df.reset_index(drop=True, inplace=True)
    # Variables for long trades
    long_position = 0  # 0 = no position, 1 = long
    long_entry_price = 0
    long_entry_date = None
    long_trades = []
    tradebook = pd.DataFrame()
    tradebook_long = pd.DataFrame()
    # tradebook_short = pd.DataFrame()
    long_trailing_stop = None
    # Variables for short trades
    # short_position = 0  # 0 = no position, -1 = short
    # short_entry_price = 0
    # short_entry_date = None
    # short_trades = []
    # short_trailing_stop = None

    can_add_long = False
    # can_add_short = False

    trade_number = 0
    lock_initial_sl = False
    first_sl = 0

    lock_initial_sl_2 = False
    first_sl_2 = 0

    pyramid_number = 0
    can_pyramid = True
    pyramid_high = 0

    for i in range(1, len(df)):
        # Entry signals for long trades
        long_trailing_stop = df.loc[i, 'EMA_20']
        # short_trailing_stop = df.loc[i, 'EMA_20']
        
        if df.loc[i, 'Buy_Signal'] and long_position == 0:
            long_position = 1
            long_entry_price = df.loc[i, 'close']
            long_entry_date = df.loc[i, 'datetime']
            long_trailing_stop = df.loc[i, 'EMA_20']
            long_initial_sl = df.loc[i, 'X_Low']

            tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
                'Trade No.': trade_number,
                'Entry_Date': df.loc[i, 'datetime'],
                'Exit_Date': None,
                'Trade_Type': 'Long',
                'Entry_Price': df.loc[i, 'close'],
                'Previous MA Value': df.loc[i-1, 'X_High'],
                'Initial SL': long_initial_sl,
                'Exit_Price': None,
                # 'Profit/Loss': None  # Filled when exited
            }])], ignore_index=True)

            if not lock_initial_sl:
                first_sl = long_initial_sl
                lock_initial_sl = True

            can_pyramid = True
            pyramid_number = 0
            pyramid_high = 0
            continue
                
        # Exit or add to long position
        if long_position == 1:

            
            # y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
            # y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
            # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
            if df.loc[i, 'low'] <= long_initial_sl:
                tradebook_long.loc[
                    (tradebook_long['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , long_initial_sl , 'Initial SL Hit']
                long_position = 0
                can_add_long = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0
            
            elif df.loc[i, 'close'] <= long_trailing_stop:
                tradebook_long.loc[
                    (tradebook_long['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'Trailing SL Hit']
                long_position = 0
                can_add_long = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0

            # elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
            #     # Previous Y Candle Low Breached, confirmation found. Add position on high break
            #     can_add_long = True

            # if can_add_long and long_position and can_pyramid:
            #     if df.loc[i, 'high'] >= y_days_high and pyramid_number < pyr_num:
            #         tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
            #             'Trade No.': trade_number,
            #             'Entry_Date': df.loc[i, 'datetime'],
            #             'Exit_Date': None,
            #             'Trade_Type': 'Add_Long',
            #             'Entry_Price': y_days_high,
            #             'Initial SL': first_sl,
            #             'Exit_Price': None,
            #             # 'Profit/Loss': None  # Filled when exited
            #         }])], ignore_index=True)
            #         can_add_long = False
            #         pyramid_number += 1
            #         can_pyramid = True
    
    tradebook = pd.concat([tradebook_long, tradebook], ignore_index=True)
    return tradebook


In [15]:
# def backtest(df, y_days):
#     # Variables for long trades
#     long_position = 0  # 0 = no position, 1 = long
#     long_entry_price = 0
#     long_entry_date = None
#     long_trades = []
#     tradebook = pd.DataFrame()
#     tradebook_long = pd.DataFrame()
#     tradebook_short = pd.DataFrame()
#     long_trailing_stop = None
#     # Variables for short trades
#     short_position = 0  # 0 = no position, -1 = short
#     short_entry_price = 0
#     short_entry_date = None
#     short_trades = []
#     short_trailing_stop = None

#     can_add_long = False
#     can_add_short = False

#     trade_number = 0

#     for i in range(len(df)):
#         # Entry signals for long trades
#         long_trailing_stop = df.loc[i, 'EMA_20']
#         short_trailing_stop = df.loc[i, 'EMA_20']
        
#         if df.loc[i, 'Buy_Signal'] and long_position == 0:
#             long_position = 1
#             long_entry_price = df.loc[i, 'close']
#             long_entry_date = df.loc[i, 'datetime']
#             long_trailing_stop = df.loc[i, 'EMA_20']
#             long_initial_sl = df.loc[i, 'X_Low']

#             tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
#                 'Trade No.': trade_number,
#                 'Entry_Date': df.loc[i, 'datetime'],
#                 'Exit_Date': None,
#                 'Trade_Type': 'Long',
#                 'Entry_Price': df.loc[i, 'close'],
#                 'Initial SL': long_initial_sl,
#                 'Exit_Price': None,
#                 # 'Profit/Loss': None  # Filled when exited
#             }])], ignore_index=True)

#         # Exit or add to long position
#         if long_position == 1:
            
#             y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
#             y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
#             # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
#             if df.loc[i, 'low'] <= long_initial_sl:
#                 tradebook_long.loc[
#                     (tradebook_long['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , long_initial_sl]
#                 long_position = 0
#                 can_add_long = False
#                 trade_number += 1
            
#             elif df.loc[i, 'close'] <= long_trailing_stop:
#                 tradebook_long.loc[
#                     (tradebook_long['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , df.loc[i, 'close']]
#                 long_position = 0
#                 can_add_long = False
#                 trade_number += 1

#             elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
#                 # Previous Y Candle Low Breached, confirmation found. Add position on high break
#                 can_add_long = True

#             if can_add_long and long_position:
#                 if df.loc[i, 'high'] >= y_days_high:
#                     tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
#                         'Trade No.': trade_number,
#                         'Entry_Date': df.loc[i, 'datetime'],
#                         'Exit_Date': None,
#                         'Trade_Type': 'Add_Long',
#                         'Entry_Price': y_days_high,
#                         'Initial SL': long_initial_sl,
#                         'Exit_Price': None,
#                         # 'Profit/Loss': None  # Filled when exited
#                     }])], ignore_index=True)
#                     can_add_long = False

#     for i in range(len(df)):
        
#         # Entry signals for long trades
#         long_trailing_stop = df.loc[i, 'EMA_20']
#         short_trailing_stop = df.loc[i, 'EMA_20']

#         # Entry signals for short trades
#         if df.loc[i, 'Sell_Signal'] and short_position == 0:
#             short_position = -1
#             short_entry_price = df.loc[i, 'close']
#             short_entry_date = df.loc[i, 'datetime']
#             short_trailing_stop = df.loc[i, 'EMA_20']
#             short_initial_sl = df.loc[i, 'X_High']

#             tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
#                 'Trade No.': trade_number,
#                 'Entry_Date': df.loc[i, 'datetime'],
#                 'Exit_Date': None,
#                 'Trade_Type': 'Short',
#                 'Entry_Price': df.loc[i, 'close'],
#                 'Initial SL': short_initial_sl,
#                 'Exit_Price': None,
#                 # 'Profit/Loss': None  # Filled when exited
#             }])], ignore_index=True)

#         if short_position == -1:
            
#             y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
#             y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
#             # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
#             if df.loc[i, 'high'] >= short_initial_sl:
#                 tradebook_short.loc[
#                     (tradebook_short['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] ,short_initial_sl]
#                 short_position = 0
#                 can_add_short = False
#                 trade_number += 1
            
#             elif df.loc[i, 'close'] >= short_trailing_stop:
#                 tradebook_short.loc[
#                     (tradebook_short['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , df.loc[i, 'close']]
#                 short_position = 0
#                 can_add_short = False
#                 trade_number += 1

#             elif (df.loc[i, 'high'] >= y_days_high) and not can_add_short:
#                 # Previous Y Candle High Breached, confirmation found. Add position on low break
#                 can_add_short = True

#             if can_add_short and short_position:
#                 if df.loc[i, 'low'] <= y_days_low:
#                     tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
#                         'Trade No.': trade_number,
#                         'Entry_Date': df.loc[i, 'datetime'],
#                         'Exit_Date': None,
#                         'Trade_Type': 'Add_Short',
#                         'Entry_Price': y_days_low,
#                         'Initial SL': short_initial_sl,
#                         'Exit_Price': None,
#                         # 'Profit/Loss': None  # Filled when exited
#                     }])], ignore_index=True)
#                     can_add_short = False
    
#     tradebook = pd.concat([tradebook_long, tradebook_short], ignore_index=True)
#     return tradebook


In [34]:
ema_length = 68
x_days = 27
# y_days = 4
pct = 0.1
RPT = 3
df = bnf_1min
df = calculate_signals(df, ema_length,x_days, pct)
# print(df.tail().to_string())
# df1 = df[df['Buy_Signal']]
# print(df1.to_string())
tb = backtest3(df)
tb = tb.sort_values(by='Entry_Date')
# variation = f'EMA: {ema_length}, X: {x_days}, Y: {y_days}, PCT: {pct}%, RPT: {RPT}%'

/tmp/ipykernel_66191/3149794742.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


In [35]:
import numpy as np
portfolio = 1250000
tb['PnL'] = np.where(
    tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
    tb['Exit_Price'] - tb['Entry_Price'],
    np.where(
        tb['Trade_Type'].isin(['Short', 'Add_Short']),  # For Short and Add_Short
        tb['Entry_Price'] - tb['Exit_Price'],
        0  # Default case if Trade_Type is something unexpected
    )
)

tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
tb['Qty'] = abs(RPT / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
# tb['Qty'] = portfolio * 8 / tb['Entry_Price']
tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
tb['Trade Year'] = tb['Entry_Date'].dt.year

In [36]:
# tb['Cumulative_PnL'] = tb['PnL w cs'].cumsum()

In [37]:
# tb.to_csv('GOLD TRADE CHECK JJPP.csv', index=False)

In [38]:
tb = tb.reset_index()
# tb

In [39]:
import matplotlib.pyplot as plt

def plot_cumulative_pnl(tb, title="Cumulative PnL Over Time"):
    if 'Cumulative_PnL' not in tb.columns:
        raise ValueError("The DataFrame must contain a 'Cumulative_PnL' column.")

    # Plot cumulative PnL
    plt.figure(figsize=(12, 3))
    plt.plot(tb.index, tb['Cumulative_PnL'], label='Cumulative PnL', color='blue', linewidth=2)
    plt.title(title, fontsize=8)
    plt.xlabel('Trade Index', fontsize=6)
    plt.ylabel('Cumulative PnL', fontsize=6)
    plt.legend(fontsize=6)
    plt.grid(alpha=0.5)
    plt.show()


In [40]:
# tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
# tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()

In [41]:
# tb_long_only = tb_long_only[tb_long_only['Trade Year'] >= 2017]

In [42]:
# plot_cumulative_pnl(tb_long_only)
# print(tb_long_only['PnL w cs'].sum())

In [43]:
# plot_cumulative_pnl(tb_short_only)
# print(tb_short_only['PnL w cs'].sum())

In [44]:
# plot_cumulative_pnl(tb)
# print(tb['PnL w cs'].sum())

In [45]:
# plot_cumulative_pnl(tb2)
# print(tb2['PnL w cs'].sum())

In [46]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [47]:
tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
tb['Trade Year'] = tb['Entry_Date'].dt.year
# tb['ROI% w cs'] = tb['ROI%']

In [48]:
stats = generate_stats(tb, '...')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,0,0,NaN,NaN,NaN,NaN,NaN,...
2018,131.8900,48,41.6667,8.9333,-1.6706,-11.7019,11.2708,...
2019,57.9944,56,33.9286,6.7489,-1.8982,-15.8124,3.6677,...
2020,122.5361,54,37.0370,9.0973,-1.7474,-12.6622,9.6773,...
2021,126.9033,60,38.3333,7.9677,-1.5231,-16.9031,7.5077,...
2022,103.4517,41,46.3415,7.8455,-2.0733,-18.2184,5.6784,...
2023,134.0144,54,29.6296,12.6967,-1.8193,-19.6061,6.8353,...
2024,47.0115,55,32.7273,6.4597,-1.8720,-19.9513,2.3563,...
2025,-10.1859,16,12.5000,2.5734,-1.0952,-12.9276,-0.7879,...
Overall,713.6156,384,35.6771,8.3630,-1.7495,-22.1200,32.2611,...


# Gold 4H X_Days High, Y_Days Low 

In [49]:
tb1 = tb[tb['Trade_Type'] == 'Long']
tb2 = tb[tb['Trade_Type'] == 'Add_Long']
tb1['ROI%'].sum() , tb2['ROI%'].sum()

(713.6155679282421, 0)

In [50]:
tb.tail(10)

,index,Trade No.,Entry_Date,Exit_Date,Trade_Type,Entry_Price,Previous MA Value,Initial SL,Exit_Price,Exit Remark,PnL,Slippage,PnL w cs,Qty,Profit_Loss INR,ROI%,Trade Year
374,374,374,2025-01-16 15:15:00+05:30,2025-01-17 09:15:00+05:30,Long,77151.1100,76914.3967,76794.0574,76794.0574,Initial SL Hit,-357.0526,15.3945,-372.4471,105.0265,-39116.8329,-3.1293,2025
375,375,375,2025-01-20 11:15:00+05:30,2025-01-21 09:15:00+05:30,Long,77245.5400,76869.7159,76712.3026,76897.2900,Trailing SL Hit,-348.2500,15.4143,-363.6643,70.3251,-25574.7448,-2.0460,2025
376,376,376,2025-01-23 09:45:00+05:30,2025-01-23 11:15:00+05:30,Long,76607.2600,76447.3407,76111.3489,76577.8000,Trailing SL Hit,-29.4600,15.3185,-44.7785,75.6184,-3386.0785,-0.2709,2025
377,377,377,2025-01-23 11:45:00+05:30,2025-01-23 14:15:00+05:30,Long,76617.3600,76416.9889,76141.4026,76583.2000,Trailing SL Hit,-34.1600,15.3201,-49.4801,78.7886,-3898.4625,-0.3119,2025
378,378,378,2025-01-24 09:15:00+05:30,2025-01-24 09:45:00+05:30,Long,76607.9700,76410.7107,76273.5704,76383.5100,Trailing SL Hit,-224.4600,15.2991,-239.7591,112.1413,-26886.8960,-2.1510,2025
379,379,379,2025-01-24 10:45:00+05:30,2025-01-24 12:15:00+05:30,Long,76796.1500,76468.7685,76308.7570,76584.8400,Trailing SL Hit,-211.3100,15.3381,-226.6481,76.9400,-17438.2980,-1.3951,2025
380,380,380,2025-01-28 11:45:00+05:30,2025-01-28 12:15:00+05:30,Long,76175.1700,76014.0826,75775.7789,76128.7300,Trailing SL Hit,-46.4400,15.2304,-61.6704,93.8929,-5790.4134,-0.4632,2025
381,381,381,2025-01-28 12:45:00+05:30,2025-01-28 14:45:00+05:30,Long,76284.7000,75958.7748,75731.9670,75959.3300,Trailing SL Hit,-325.3700,15.2244,-340.5944,67.8447,-23107.5238,-1.8486,2025
382,382,382,2025-01-29 09:45:00+05:30,2025-02-03 09:15:00+05:30,Long,76241.7600,75902.8648,75704.1107,76893.1200,Trailing SL Hit,651.3600,15.3135,636.0465,69.7481,44363.0188,3.5490,2025
383,383,383,2025-02-04 09:15:00+05:30,2025-02-07 12:45:00+05:30,Long,77930.0900,77424.3293,77185.0793,77818.5200,Trailing SL Hit,-111.5700,15.5749,-127.1449,50.3348,-6399.8168,-0.5120,2025


In [51]:
tb.to_csv('Turtle_SENSEX.csv')

In [33]:
# tb_long_only.to_csv('GOLD Range JJPP.csv', index=False)

In [147]:
# ema_length = 55
# x_days = 28
# y_days = 7
# pct = 1.05
# RPT = 5

pct_range = [0.1, 0.25]
RPT_ = 3
stats_dictionary = {}

for ema_length in range(20, 81, 2):
    for x_days in range(3, 37, 3):
            for pct in pct_range:
                # RPT = 5
                variation = f'EMA:{ema_length}, X:{x_days}, PCT:{pct}'
                print(variation)
                df = bnf_1min
                df = calculate_signals(df, ema_length=ema_length, x_days=x_days, pct=pct)
                tb = backtest3(df)
                if len(tb) > 0:
                    tb = tb.sort_values(by='Entry_Date')
                    portfolio = 10000000
                    tb['PnL'] = np.where(
                        tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
                        tb['Exit_Price'] - tb['Entry_Price'],
                        np.where(
                            tb['Trade_Type'].isin(['Short', 'Add_Short']),  # For Short and Add_Short
                            tb['Entry_Price'] - tb['Exit_Price'],
                            0  # Default case if Trade_Type is something unexpected
                        )
                    )
                    
                    tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
                    tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
                    tb['Qty'] = abs(RPT_ / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
                    tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
                    tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
                    tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
                    tb['Trade Year'] = tb['Entry_Date'].dt.year
                    # tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
                    # tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()
                    # tb_short_only = tb[(tb['Trade_Type'] == 'Short') | (tb['Trade_Type'] == 'Add_Short')].copy()
                    # tb_short_only['Cumulative_PnL'] = tb_short_only['PnL w cs'].cumsum()
                    
                    stats1 = generate_stats(tb, variation)
                    for x, y in stats1.items():
                        if x > 30:
                            # print('Long Only')
                            print(pd.DataFrame(y).to_string())
                            stats_dictionary[x] = y
                        break

EMA:20, X:3, PCT:0.1


/tmp/ipykernel_38421/3149794742.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:20, X:3, PCT:0.25


/tmp/ipykernel_38421/3149794742.py:118: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Initial SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:20, X:6, PCT:0.1


...

/tmp/ipykernel_38421/3149794742.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:80, X:36, PCT:0.1


/tmp/ipykernel_38421/3149794742.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:80, X:36, PCT:0.25


/tmp/ipykernel_38421/3149794742.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


# SENSEX 30min

In [149]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
for x, y in sorted_stats.items():
    print(y.to_string())

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio              Variation
2017            0            0      NaN                   NaN                 NaN          NaN          NaN  EMA:68, X:27, PCT:0.1
2018     131.8900           48  41.6667                8.9333             -1.6706     -11.7019      11.2708  EMA:68, X:27, PCT:0.1
2019      57.9944           56  33.9286                6.7489             -1.8982     -15.8124       3.6677  EMA:68, X:27, PCT:0.1
2020     122.5361           54  37.0370                9.0973             -1.7474     -12.6622       9.6773  EMA:68, X:27, PCT:0.1
2021     126.9033           60  38.3333                7.9677             -1.5231     -16.9031       7.5077  EMA:68, X:27, PCT:0.1
2022     103.4517           41  46.3415                7.8455             -2.0733     -18.2184       5.6784  EMA:68, X:27, PCT:0.1
2023     134.0144           54  29.6296               12.6967             -1.8193  